In [2]:
PATH_FILE_PREPROCESSED = './results/ds.parquet'
PATH_FILE_ELAPSE_TIME = './results/elapse_time.csv'
PATH_PREFIX_MODEL_LDA = './results/models_lda/'
PATH_PREFIX_MODEL_BERTOPIC = './results/models_bertopic/'
LDA_N_TRIAL = 1 # FIXME: should be 100 fr fr
USE_GPU = False

In [3]:
import pandas as pd

ds = pd.read_parquet(PATH_FILE_PREPROCESSED)
elapse_time = pd.read_csv(PATH_FILE_ELAPSE_TIME)

# LDA

In [4]:
from time import time

from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from utils import get_diversity, get_topics_lda
import optuna

optuna.logging.set_verbosity(optuna.logging.WARNING)

def lda(docs):
    docs = docs.dropna()
    dictionary = Dictionary(docs)
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    def objective(trial, get_lda=False):
        num_topics = trial.suggest_int('num_topics', 5, 100)
        alpha = trial.suggest_categorical('alpha_categorical', ['symmetric', 'asymmetric', 'scalar'])
        eta = trial.suggest_categorical('eta_categorical', ['symmetric', 'auto', 'scalar'])
        if alpha == 'scalar':
            alpha = trial.suggest_float('alpha', 0.01, 1)
        if eta == 'scalar':
            eta = trial.suggest_float('eta', 0.01, 1)
        model = LdaMulticore(
            corpus=corpus,
            id2word=dictionary,
            num_topics=num_topics,
            alpha=alpha,
            eta=eta,
        )
        if get_lda:
            return model
        c = CoherenceModel(model, texts=docs, dictionary=dictionary, coherence='c_v')
        cs = c.get_coherence()
        ts = get_diversity(get_topics_lda(model, dictionary))
        return cs * ts

    t_start = time()
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=LDA_N_TRIAL)

    best_trial = study.best_trial
    best_lda = objective(best_trial, get_lda=True)
    t_end = time()

    return {
        'model': best_lda,
        'time': t_end - t_start,
    }

In [5]:
from tqdm import tqdm

def save_models_lda():
    r_time = []
    for v in (tds := tqdm(ds.columns)):
        tds.set_description(f'Training {v}')
        if 'B' in v: 
            r_time.append(0)
            continue
        model = lda(ds[v])
        model['model'].save(f'{PATH_PREFIX_MODEL_LDA}{v}')
        r_time.append(model['time'])
    elapse_time['lda_training'] = pd.Series(r_time)

In [6]:
save_models_lda()

Training dC:   0%|          | 0/110 [00:00<?, ?it/s]

Training DCSWP: 100%|██████████| 110/110 [12:09<00:00,  6.63s/it]


In [7]:
elapse_time

,variant,nlp,preprocessing,lda_training
0,dB,212.552485,0.001777,0.000000
1,DB,212.552485,0.001778,0.000000
2,dC,212.552485,0.043353,8.620242
3,DC,212.552485,0.043354,6.784524
4,dG,212.552485,0.035941,6.479580
...,...,...,...,...
105,DCSNP,212.552485,0.759058,7.060609
106,dCSWG,212.552485,0.640932,5.686913
107,DCSWG,212.552485,0.640937,12.209638
108,dCSWP,212.552485,0.627230,4.129492


# BERTopic

In [8]:
from time import time

import pandas as pd
from bertopic import BERTopic
# from cuml.cluster import HDBSCAN
# from cuml.manifold import UMAP
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline


def save_models_bertopic():
    r_time = []
    for v in (tds := tqdm(ds.columns)):
        tds.set_description(f'Training {v}')
        if 'T' in v: 
            r_time.append(0)
            continue
        t_start = time()

        docs = ds[v].dropna()
        docs = [' '.join(doc) for doc in docs] if 'B' not in v else docs

        if USE_GPU:
            pass # FIXME: buy gpu 💀
            # umap_model = UMAP(n_components=5, n_neighbors=15, min_dist=0.0)
            # hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, prediction_data=True)
            # bertopic = BERTopic(language='multilingual', umap_model=umap_model, hdbscan_model=hdbscan_model)
        else:
            pipe = make_pipeline(
                TfidfVectorizer(),
                TruncatedSVD(100)
            )
            bertopic = BERTopic(embedding_model=pipe)
        model = bertopic.fit(docs)

        t_end = time()
        model.save(
            f"{PATH_PREFIX_MODEL_BERTOPIC}{v}",
            serialization="safetensors",
            save_ctfidf=True,
        )
        r_time.append(t_end - t_start)
    elapse_time['bertopic_training'] = pd.Series(r_time)

In [9]:
save_models_bertopic()

Training dB:   0%|          | 0/110 [00:00<?, ?it/s]

Training DCSWP: 100%|██████████| 110/110 [11:27<00:00,  6.25s/it]


In [10]:
elapse_time

,variant,nlp,preprocessing,lda_training,bertopic_training
0,dB,212.552485,0.001777,0.000000,18.465299
1,DB,212.552485,0.001778,0.000000,3.485061
2,dC,212.552485,0.043353,8.620242,8.633718
3,DC,212.552485,0.043354,6.784524,4.135852
4,dG,212.552485,0.035941,6.479580,9.766098
...,...,...,...,...,...
105,DCSNP,212.552485,0.759058,7.060609,4.202968
106,dCSWG,212.552485,0.640932,5.686913,8.407741
107,DCSWG,212.552485,0.640937,12.209638,4.086326
108,dCSWP,212.552485,0.627230,4.129492,8.897246


In [11]:
elapse_time.to_csv(PATH_FILE_ELAPSE_TIME, index=False)